In [2]:
# Import packages
import os
import geopandas as gpd
import numpy as np
from osgeo import gdal
import rasterio
import rasterio.mask
#import netCDF4 as nc

In [6]:
os.chdir('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\geospatial_internship\\datasets')
shape_telangana = gpd.read_file('telangana_shapefile.geojson')
shape_telangana = shape_telangana.explode()

C:\Users\Jesse\AppData\Local\Temp\ipykernel_1608\1639596723.py:3: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  shape_telangana = shape_telangana.explode()


In [8]:
path = 'c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires\\BA_data_pre\\c_gls_BA300_202205310000_GLOBE_S3_V1.2.1.nc'
#path2 = 'c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires\\test.tiff'

In [10]:
# Print some metadata related to the Leaf Area Index
#nc_metadata = nc.Dataset(path)
#nc_metadata['NOBS']

<class 'netCDF4._netCDF4.Variable'>
uint8 NOBS(time, lat, lon)
    _FillValue: 255
    standard_name: burned_area_fraction number_of_observations
    grid_mapping: crs
    units: 
    valid_range: [ 0 44]
    flag_values: [254 255]
    flag_meanings: Water Missing
    missing_value: 255
unlimited dimensions: time
current shape = (1, 47040, 120960)
filling on

In [11]:
# Import the netCDF data for Leaf Area Index
nc_file = gdal.Open('NETCDF:c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires\\BA_data_pre\\c_gls_BA300_202205310000_GLOBE_S3_V1.2.1.nc:NOBS')
# Convert the netCDF to Geotiff file
tif_file = gdal.Translate('c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires\\BA_data_pre\\ba_tiff.tif', nc_file)

In [13]:
# In the next two code blocks, we crop the Geotiff file to Leaf Area Index in Telangana
ba_data = "c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires\\BA_data_pre\\ba_tiff.tif"
with rasterio.open(ba_data) as src:
    out_image, out_transform = rasterio.mask.mask(src, shape_telangana.geometry, crop=True)
    out_meta = src.meta

In [14]:
out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

# Save the Geotiff file to Leaf Area Index in Telangana
with rasterio.open("c:\\Users\\Jesse\\OneDrive\\Documenten\\Master BAOR\\Thesis\\GitHub\\dicra\\analytics\\notebooks\\crop_fires\\BA_data_pre\\ba_cropped_tiff.tif", "w", **out_meta) as dest:
    dest.write(out_image)
    